In [1]:
library(Rcrawler)
library(foreach)
library(doParallel)
library(scales)
library(qdapRegex)

Loading required package: iterators

Loading required package: parallel



In [2]:
Sys.setlocale("LC_CTYPE", locale="Chinese")

[1] "Chinese (Simplified)_China.936"

In [3]:
# parallel processing
cores=detectCores()
cl <- makeCluster(cores[1]-1) #not to overload your computer
registerDoParallel(cl)

In [4]:
# CSS patterns identified from website
pattern <- c(".content_table_R a", 
             ".content_table_R td:nth-child(4)",
             ".content_table_R td:nth-child(5)",
             ".content_table_R td:nth-child(6)",
             ".content_table_R td:nth-child(7)",
             ".content_table_R td:nth-child(8)")

# Names for each field collected
pattern_names <- c("id", 
                   "wins",
                   "battles",
                   "fleet",
                   "squad",
                   "activity")

scraper <- function(page){
  target_url <- paste("https://www.navyfield.com/Community/Ranking/List.aspx?page=",
                      toString(page),"&server=1&category=4&searchType=1&searchValue=", sep = "")
  web_content <- Rcrawler::ContentScraper(Url=target_url,
                                CssPatterns = pattern,
                                PatternsName = pattern_names, 
                                ManyPerPattern = TRUE, 
                                astext = TRUE, 
                                asDataFrame = FALSE)
  output <- as.data.frame(web_content)
  return(output)
}

rank_data <- foreach(i=1:500, .combine=rbind) %dopar% {
  tempMatrix = scraper(i) 
  tempMatrix #Equivalent to finalMatrix = rbind(finalMatrix, tempMatrix)
}

In [5]:
hof_pattern <- c(".rnum", 
             ".content_table_H td:nth-child(4)",
             ".content_table_H td:nth-child(5)",
             ".content_table_H td:nth-child(6)",
             ".content_table_H td:nth-child(7)")

hof_pattern_names <- c("id", 
                   "wins_total",
                   "fleet",
                   "squad",
                   "activity")

hof_scraper <- function(page){
    target_url <- paste("https://www.navyfield.com/Community/HallofFame/List.aspx?page=",
                        toString(page),"&server=&sort=1&searchType=1&searchValue=", sep = "")
    web_content <- Rcrawler::ContentScraper(Url=target_url,
                                  CssPatterns = hof_pattern,
                                  PatternsName = hof_pattern_names, 
                                  ManyPerPattern = TRUE, 
                                  astext = TRUE, 
                                  asDataFrame = FALSE)
    output <- as.data.frame(web_content)
    return(output)
}


hof_data <- foreach(i=1:7, .combine=rbind) %dopar% {
  tempMatrix = hof_scraper(i) 
  tempMatrix 
}

hof_data$activity = rm_between(hof_data$activity, '(', ')', extract=TRUE)
hof_data <- as.data.frame(lapply(hof_data, function(x) {
  gsub("\u00A0", "", x) 
})) 
hof_data$wins <- sub("/.*", "", hof_data$wins_total)
hof_data$battles <- sub(".*/", "", hof_data$wins_total)
hof_clean <- subset(hof_data, select = c(id, fleet:battles))

In [6]:
nf_data <- rbind(rank_data, hof_clean)
nf_data$id <- trimws(nf_data$id, which = c("both"))
nf_data$fleet <- trimws(nf_data$fleet, which = c("both"))
nf_data$squad <- trimws(nf_data$squad, which = c("both"))
nf_data$activity <- trimws(nf_data$activity, which = c("both"))
nf_data$activity <- gsub(" ", "", as.character(nf_data$activity))
nf_data$activity <- gsub("PST", "", as.character(nf_data$activity))
nf_data$wins <- as.numeric(gsub(",", "", as.character(nf_data$wins)))
nf_data$battles <- as.numeric(gsub(",", "", as.character(nf_data$battles)))
nf_data$win_rate <- nf_data$wins/nf_data$battles

In [7]:
nf <- file("C:/Users/Felix/Box Sync/UW/R/nf.csv", encoding = "UTF-8")
write.csv(nf_data, file = nf, row.names = FALSE)

In [8]:
# See results
substrRight <- function(x, n){
  as.numeric(substr(x, nchar(x)-n+1, nchar(x)))
}

substrLeft <- function(x, n){
  as.numeric(substr(x, 1, n))
}

today <- as.character(Sys.Date())
current_year <- as.numeric(substr(today,1,4))
current_month <- as.numeric(substr(today,6,7))

last_three_months <- function(m,y){
    if (m < 4) {
        a = m + 9
        b = y - 1
    }
    else if (m > 3) {
        a = m - 3
        b = y
    }
    return(c(a,b))
}

fleet_power <- function(fleet_name){
    fleet_data <- subset(nf_data, fleet==fleet_name)
    attach(fleet_data)
    fleet_best <- fleet_data[order(-win_rate),]
    fleet_best$win_rate <- percent(fleet_best$win_rate, accuracy = 0.1)
    detach(fleet_data)
    if (current_month > 3){
        return(head(subset(fleet_best,battles > 5000 & substrRight(fleet_best$activity,4)==last_three_months(current_month,current_year)[2] & substrLeft(fleet_best$activity,2)>last_three_months(current_month,current_year)[1]),20))
    } else if (current_month < 4){
        return(head(subset(fleet_best,battles > 5000 & 
                           (substrRight(fleet_best$activity,4)>last_three_months(current_month,current_year)[2] | 
                           (substrRight(fleet_best$activity,4)==last_three_months(current_month,current_year)[2] & substrLeft(fleet_best$activity,2)>=last_three_months(current_month,current_year)[1])
                           )
                          ),20))
    }  
}

squad_power <- function(fleet_name){
    fleet_data <- subset(nf_data, squad==fleet_name)
    attach(fleet_data)
    fleet_best <- fleet_data[order(-win_rate),]
    fleet_best$win_rate <- percent(fleet_best$win_rate, accuracy = 0.1)
    detach(fleet_data)
    if (current_month > 3){
        return(head(subset(fleet_best,battles > 5000 & substrRight(fleet_best$activity,4)==last_three_months(current_month,current_year)[2] & substrLeft(fleet_best$activity,2)>last_three_months(current_month,current_year)[1]),20))
    } else if (current_month < 4){
        return(head(subset(fleet_best,battles > 5000 & 
                           (substrRight(fleet_best$activity,4)>last_three_months(current_month,current_year)[2] | 
                           (substrRight(fleet_best$activity,4)==last_three_months(current_month,current_year)[2] & substrLeft(fleet_best$activity,2)>=last_three_months(current_month,current_year)[1])
                           )
                          ),20))
    }  
}

In [9]:
# NF players that win and/or disconnect most of the time (<_<)
# Only players active in last 3 months and have had 5000+ battles on record are listed
attach(nf_data)
nf_best <- nf_data[order(-win_rate),]
nf_best$win_rate <- percent(nf_best$win_rate, accuracy = 0.1)
detach(nf_data)
if (current_month > 3){
    head(subset(nf_best,battles > 5000 & substrRight(nf_best$activity,4)==last_three_months(current_month,current_year)[2] & substrLeft(nf_best$activity,2)>=last_three_months(current_month,current_year)[1]),20)
} else if (current_month < 4){
    head(subset(nf_best,battles > 5000 & 
                (substrRight(nf_best$activity,4)==last_three_months(current_month,current_year)[2] & substrLeft(nf_best$activity,2)>=last_three_months(current_month,current_year)[1]) |
                substrRight(nf_best$activity,4)>last_three_months(current_month,current_year)[2]
               ),20)
}

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1100,joooooo,6283,7868,REGALIA,-,07.21.2021,79.9%
928,nuke666,7065,9005,Russian Alliance,AborzSQ,07.04.2021,78.5%
1727,baozi77,4332,5611,-,-,07.20.2021,77.2%
1459,pr_ingress,5033,6531,TEAM PRO BOTS,-,07.20.2021,77.1%
242,jedizorro,13186,17294,Dango Daikazoku,-,07.20.2021,76.2%
10022,yyss2221,20562,27461,海风舰队,海風奔馳,07.20.2021,74.9%
879,betep,7287,9941,Russian Alliance,Nakhimov,07.21.2021,73.3%
247,k435345,13128,18023,-,-,07.04.2021,72.8%
243,greattak,13184,18291,-,-,07.20.2021,72.1%


In [39]:
# 深海舰队
fleet_power('深海舰队')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1265,b911753382a,5684,9721,深海舰队,-,07.20.2021,58.5%
1498,zcx19740112,4937,8454,深海舰队,深渊,05.06.2021,58.4%
1846,chalex001,4084,7433,深海舰队,-,07.20.2021,54.9%
1245,erissoft,5749,10676,深海舰队,深渊,07.08.2021,53.8%
1254,wjsh2020,5725,10946,深海舰队,-,07.21.2021,52.3%
1491,guanguanktv,4950,9492,深海舰队,深蓝战队,07.20.2021,52.1%
2114,zbyzw2001,3440,6643,深海舰队,深渊,07.21.2021,51.8%
2610,xingzi1982,2649,5155,深海舰队,-,07.08.2021,51.4%
2098,ddra721027,3469,6751,深海舰队,-,07.19.2021,51.4%


In [40]:
# 兔子舰队
fleet_power('兔子.CHN.')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1098,la2gajaltmot,6288,9133,兔子.CHN.,World,07.21.2021,68.8%
145,tskspree,15668,22999,兔子.CHN.,World,07.20.2021,68.1%
3,laoji,37700,55611,兔子.CHN.,-,07.20.2021,67.8%
1592,hina272k,4657,6999,兔子.CHN.,Bravery,07.20.2021,66.5%
236,dreamcaca0,13356,20590,兔子.CHN.,World,07.20.2021,64.9%
430,zilong001,10621,17009,兔子.CHN.,pretender,07.04.2021,62.4%
331,aircraftcarrier,11746,18847,兔子.CHN.,Hermit,07.20.2021,62.3%
1208,liu001001,5858,9536,兔子.CHN.,讨债办,06.29.2021,61.4%
1651,imanoob,4505,7336,兔子.CHN.,pretender,07.08.2021,61.4%


In [41]:
# 荣誉舰队
fleet_power('新荣誉')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1595,guxiaoqiang,4654,7898,新荣誉,虎贲军,07.20.2021,58.9%
2053,hsjj005,3581,6184,新荣誉,天下,05.29.2021,57.9%
1565,zeon0079,4728,8333,新荣誉,海狼团,07.21.2021,56.7%
2271,leviathan179,3128,5523,新荣誉,海狼团,07.19.2021,56.6%
1917,zwl1972,3915,7139,新荣誉,-,07.20.2021,54.8%
2474,a1843452343,2829,5258,新荣誉,虎贲军,06.04.2021,53.8%
701,haksyx2003,8365,15613,新荣誉,海狼团,07.20.2021,53.6%
2528,qwe3147,2757,5146,新荣誉,海狼团,07.20.2021,53.6%
2488,godissatan,2813,5256,新荣誉,虎贲军,07.21.2021,53.5%


In [42]:
# 中华联合舰队
fleet_power('中华联合舰队')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1192,liming,5908,9096,中华联合舰队,-,07.21.2021,65.0%
1220,nfship001,5833,9772,中华联合舰队,-,07.21.2021,59.7%
1815,ssdy2736,4129,6944,中华联合舰队,海魂,06.06.2021,59.5%
310,hsx330136,12018,20486,中华联合舰队,-,07.21.2021,58.7%
437,daiyinjj,10570,18022,中华联合舰队,中华联合一队,07.20.2021,58.7%
67,ak47ok,20025,34147,中华联合舰队,中华联合二队,07.19.2021,58.6%
1646,qq95568466,4517,7879,中华联合舰队,-,07.20.2021,57.3%
1309,lee1,5548,9678,中华联合舰队,-,07.04.2021,57.3%
451,down360,10380,18209,中华联合舰队,-,07.19.2021,57.0%


In [43]:
# The Guardians Fleet
fleet_power('Dango Daikazoku')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
242,jedizorro,13186,17294,Dango Daikazoku,-,07.20.2021,76.2%
648,hanyizhou,8624,13137,Dango Daikazoku,-,07.04.2021,65.6%
1785,kf222,4204,6935,Dango Daikazoku,Dragon,07.19.2021,60.6%
386,yixi,11147,18911,Dango Daikazoku,Dragon,07.19.2021,58.9%
307,bibber,12100,20692,Dango Daikazoku,-,07.20.2021,58.5%
1556,yupeng1982,4756,8137,Dango Daikazoku,Dragon,06.27.2021,58.4%
10042,iloveasu,7898,13532,Dango Daikazoku,-,07.21.2021,58.4%
408,ttlz,10912,18761,Dango Daikazoku,Dragon,07.02.2021,58.2%
406,skybear991,10925,18821,Dango Daikazoku,Dragon,07.12.2021,58.0%


In [44]:
# Team Pro Bots
fleet_power('TEAM PRO BOTS')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1459,pr_ingress,5033,6531,TEAM PRO BOTS,-,07.20.2021,77.1%
944,vosu,6958,9696,TEAM PRO BOTS,TheBestOfBest,07.18.2021,71.8%
347,ded111,11586,16678,TEAM PRO BOTS,$CHMUTZ,07.21.2021,69.5%
545,portunus,9406,14009,TEAM PRO BOTS,$CHMUTZ,07.18.2021,67.1%
33,painlesskiss,24105,36766,TEAM PRO BOTS,-,07.21.2021,65.6%
244,quendil,13173,21292,TEAM PRO BOTS,-,07.20.2021,61.9%
747,schoko,8110,13345,TEAM PRO BOTS,$CHMUTZ,07.21.2021,60.8%
334,hollowman_eu,11706,20194,TEAM PRO BOTS,$CHMUTZ,07.20.2021,58.0%
1071,bestofchill,6412,11131,TEAM PRO BOTS,$CHMUTZ,07.21.2021,57.6%


In [45]:
# BAMF Fleet
fleet_power('BAMF FLEET')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1423,kenvahnz,5161,7440,BAMF FLEET,NERFED,07.20.2021,69.4%
692,ooh_kill_em,8424,12155,BAMF FLEET,Legacy,07.18.2021,69.3%
81,tappo01,19008,27987,BAMF FLEET,DooM,07.14.2021,67.9%
196,koll_,14390,21546,BAMF FLEET,Legacy,07.18.2021,66.8%
10108,lordfail,8977,13625,BAMF FLEET,DooM,06.25.2021,65.9%
1,ms06zaku,60190,92515,BAMF FLEET,BULLS,07.20.2021,65.1%
336,saladrex,11670,18448,BAMF FLEET,NERFED,06.09.2021,63.3%
107,wilsonjordan,16874,26900,BAMF FLEET,#Karma,05.01.2021,62.7%
223,admiralkoth,13731,21950,BAMF FLEET,NERFED,05.15.2021,62.6%


In [46]:
# Russian Alliance
fleet_power('Russian Alliance')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
928,nuke666,7065,9005,Russian Alliance,AborzSQ,07.04.2021,78.5%
879,betep,7287,9941,Russian Alliance,Nakhimov,07.21.2021,73.3%
1761,adorius,4256,6717,Russian Alliance,AborzSQ,07.11.2021,63.4%
463,osn3179,10261,16285,Russian Alliance,AborzSQ,07.18.2021,63.0%
810,xxvetalxx,7731,12340,Russian Alliance,Makarov,07.17.2021,62.6%
688,syava_mehan,8436,13676,Russian Alliance,-,07.18.2021,61.7%
1454,varangian,5043,8200,Russian Alliance,-,07.21.2021,61.5%
2155,gother_nfru,3354,5622,Russian Alliance,-,07.06.2021,59.7%
137,reisereise,15979,26909,Russian Alliance,AborzSQ,07.11.2021,59.4%


In [47]:
# 中国蓝舰队
fleet_power('中国蓝舰队')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1693,zjtz000000,4416,7291,中国蓝舰队,☆ ☆ ☆,07.21.2021,60.6%
714,wang82840506,8299,13702,中国蓝舰队,☆ ☆ ☆,07.20.2021,60.6%
1032,jovacn,6575,11442,中国蓝舰队,★ ☆ ★,07.20.2021,57.5%
1790,f9977,4199,7405,中国蓝舰队,★ ★ ★,07.15.2021,56.7%
375,wang811019,11251,19851,中国蓝舰队,-,07.14.2021,56.7%
245,kxjh,13170,23707,中国蓝舰队,★ ★ ☆,07.16.2021,55.6%
1925,e9977,3895,7031,中国蓝舰队,☆ ☆ ☆,07.15.2021,55.4%
1387,solo3162,5261,9568,中国蓝舰队,★ ★ ★,07.19.2021,55.0%
131,hz111111,16114,29315,中国蓝舰队,☆ ☆ ☆,07.21.2021,55.0%


In [48]:
# 北方舰队
fleet_power('北方舰队')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1500,dongbeiwang,4934,7601,北方舰队,Odin,07.20.2021,64.9%
454,bin2014,10323,16662,北方舰队,THOR,07.20.2021,62.0%
491,bfhjzrb1319,9983,16654,北方舰队,-,07.20.2021,59.9%
2063,langhero,3566,6186,北方舰队,THOR,07.15.2021,57.6%
2014,andy_366,3681,6436,北方舰队,Odin,07.20.2021,57.2%
1506,gbsb8,4911,8624,北方舰队,北方天骄,07.08.2021,56.9%
964,abcdkent,6876,12386,北方舰队,THOR,07.20.2021,55.5%
1207,playab,5858,10613,北方舰队,-,07.17.2021,55.2%
2331,yaoyaopi,3046,5682,北方舰队,THOR,07.19.2021,53.6%


In [49]:
# Ostseepiraten Fleet
fleet_power('Ostseepiraten')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
181,steuben_eu,14600,22575,Ostseepiraten,Stoertebeker,05.02.2021,64.7%
2158,ikken3,3350,5506,Ostseepiraten,P_Beneke,07.20.2021,60.8%
628,ikken4,8778,14980,Ostseepiraten,P_Beneke,07.20.2021,58.6%
10098,pamel,10255,17703,Ostseepiraten,-,07.07.2021,57.9%
163,bluestar_eu,15105,26230,Ostseepiraten,corsair,07.21.2021,57.6%
1294,kpt_jack,5586,9993,Ostseepiraten,Stoertebeker,07.01.2021,55.9%
1222,hippo83,5827,10463,Ostseepiraten,P_Beneke,05.23.2021,55.7%
652,jolly_roger2,8606,15645,Ostseepiraten,Rum_u_Ehre,07.20.2021,55.0%
1094,flausmaus,6311,11475,Ostseepiraten,Rum_u_Ehre,07.04.2021,55.0%


In [50]:
# REGALIA
fleet_power('REGALIA')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1100,joooooo,6283,7868,REGALIA,-,07.21.2021,79.9%
2166,deredador,3334,5848,REGALIA,-,07.20.2021,57.0%
498,unosco,9866,18502,REGALIA,-,07.18.2021,53.3%
520,deseo5,9634,18232,REGALIA,-,07.18.2021,52.8%
1060,translei,6437,12436,REGALIA,-,07.18.2021,51.8%
2543,entroyook,2736,5327,REGALIA,-,07.19.2021,51.4%
1162,yoshyy,6004,11770,REGALIA,-,07.19.2021,51.0%
219,loquic12,13799,27138,REGALIA,Renovation,07.21.2021,50.8%
1316,shyk,5530,10903,REGALIA,-,07.18.2021,50.7%


In [51]:
# ARMADA CHILENA
fleet_power('ARMADA CHILENA')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
178,makuvex,14663,27188,ARMADA CHILENA,-,07.18.2021,53.9%
409,lmario666,10901,20789,ARMADA CHILENA,-,07.04.2021,52.4%
1749,carlosii,4273,8261,ARMADA CHILENA,"O""Higgins",07.03.2021,51.7%
1314,xeloborg89,5535,10792,ARMADA CHILENA,-,05.27.2021,51.3%
383,klython1,11196,21890,ARMADA CHILENA,-,07.18.2021,51.1%
1722,saharas,4343,8634,ARMADA CHILENA,-,07.18.2021,50.3%
176,kudoalex,14696,29306,ARMADA CHILENA,"O""Higgins",07.20.2021,50.1%
2367,jecho,2988,5969,ARMADA CHILENA,-,07.18.2021,50.1%
526,kiros,9596,19211,ARMADA CHILENA,-,07.18.2021,50.0%


In [52]:
# Flota Chile
fleet_power('Flota Chile')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
185,cuadrad,14580,26167,Flota Chile,-,07.21.2021,55.7%
1191,kulebrakai,5909,10821,Flota Chile,-,07.18.2021,54.6%
372,xaloooo,11276,21167,Flota Chile,-,07.18.2021,53.3%
1091,gabriel_03,6332,11966,Flota Chile,Nanox,06.22.2021,52.9%
1566,breaka,4726,8932,Flota Chile,-,07.21.2021,52.9%
1093,chilotecocoa,6319,11944,Flota Chile,-,07.18.2021,52.9%
589,nitroxx,9053,17183,Flota Chile,ALDEA,06.27.2021,52.7%
754,elite_,8088,15441,Flota Chile,-,07.19.2021,52.4%
305,azsnar,12155,23207,Flota Chile,-,07.19.2021,52.4%


In [53]:
# Imperium
fleet_power('Imperium')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
71,aildinchen,19550,33081,Imperium,Aggressor,07.10.2021,59.1%
579,delta56,9165,15824,Imperium,Veterans,07.21.2021,57.9%
647,ssepan,8646,15043,Imperium,-,07.21.2021,57.5%
50,slayer_z,22225,39255,Imperium,MFDM,07.02.2021,56.6%
1268,nbarloya,5681,10136,Imperium,Aggressor,07.21.2021,56.0%
763,sema433,8026,14539,Imperium,Veterans,07.21.2021,55.2%
102,tmac2,17297,31425,Imperium,-,05.16.2021,55.0%
533,hussy,9543,17556,Imperium,MFDM,07.19.2021,54.4%
739,benbonebone,8192,15251,Imperium,-,07.20.2021,53.7%


In [54]:
# Armored Angels
fleet_power('Armored Angels')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
355,rainbow112,11479,18332,Armored Angels,Titans SQ,07.20.2021,62.6%
528,nager,9588,15876,Armored Angels,-,05.08.2021,60.4%
240,wishbonej,13230,23225,Armored Angels,-,05.14.2021,57.0%
173,will211,14784,27852,Armored Angels,-,07.20.2021,53.1%
199,hideship12,14235,26991,Armored Angels,Titans SQ,06.26.2021,52.7%
601,zemex,8976,17125,Armored Angels,Titans SQ,07.21.2021,52.4%
507,admstef,9759,18797,Armored Angels,-,07.07.2021,51.9%
1047,wawa300,6477,12658,Armored Angels,Titans SQ,07.18.2021,51.2%
122,djr63,16381,32061,Armored Angels,Saintz SQ,07.04.2021,51.1%


In [55]:
# HELL BOYS
fleet_power('HELL BOYS')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1144,_vicon_,6080,9802,HELL BOYS,ZETAS,06.29.2021,62.0%
22,altsein,26005,44322,HELL BOYS,-,07.18.2021,58.7%
146,hitachi88,15618,26816,HELL BOYS,ZETAS,07.18.2021,58.2%
29,patatoo,24588,42484,HELL BOYS,-,07.18.2021,57.9%
198,det_ka,14263,24920,HELL BOYS,ZETAS,07.18.2021,57.2%
335,dakota775,11675,20512,HELL BOYS,ZETAS,06.06.2021,56.9%
518,andr2776,9677,17216,HELL BOYS,ZETAS,07.18.2021,56.2%
426,xelementx,10676,19247,HELL BOYS,-,07.19.2021,55.5%
767,fenix69,7996,14744,HELL BOYS,LIONS,07.18.2021,54.2%


In [27]:
# Reiwa Fleet
fleet_power('Reiwa Fleet')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
2159,y4427,3346,5091,Reiwa Fleet,-,07.19.2021,65.7%
2124,wanco2121,3403,5523,Reiwa Fleet,-,05.22.2021,61.6%
2235,aituki,3193,5186,Reiwa Fleet,-,07.17.2021,61.6%
104,zatti,17182,27968,Reiwa Fleet,-,07.20.2021,61.4%
1055,sigemori,6448,12035,Reiwa Fleet,-,06.30.2021,53.6%
945,captainmaru,6956,13175,Reiwa Fleet,-,07.20.2021,52.8%
2570,cx817us,2706,5162,Reiwa Fleet,-,07.21.2021,52.4%
2148,wakasa3n,3368,6569,Reiwa Fleet,-,07.03.2021,51.3%
52,osuka,22059,43164,Reiwa Fleet,-,07.19.2021,51.1%


In [28]:
# Jedi Fleet
fleet_power('Jedi Fleet')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
12,jackaljl,29259,49480,Jedi Fleet,-,07.20.2021,59.1%
1887,sickman,4003,7012,Jedi Fleet,-,05.22.2021,57.1%
1800,black101,4169,7824,Jedi Fleet,-,06.13.2021,53.3%
367,oopsitorpedu,11337,21664,Jedi Fleet,-,05.04.2021,52.3%
1080,rutgersrocks,6372,12369,Jedi Fleet,-,05.21.2021,51.5%
1898,hotshootgg,3975,8090,Jedi Fleet,-,06.19.2021,49.1%
1561,langan,4731,10003,Jedi Fleet,-,06.22.2021,47.3%
1982,tk0rahl,3775,7996,Jedi Fleet,Sith Empire,07.08.2021,47.2%
2950,tacnuke,2222,5252,Jedi Fleet,-,06.23.2021,42.3%


In [29]:
# TeamKorea
fleet_power('TeamKorea')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
2291,mndusers,3096,5236,TeamKorea,Korea,07.20.2021,59.1%
410,timmerman,10874,18819,TeamKorea,-,07.20.2021,57.8%
1652,seedream,4503,8517,TeamKorea,-,07.11.2021,52.9%
1015,torabin,6647,12634,TeamKorea,-,07.20.2021,52.6%
1885,bnbchd,4006,7624,TeamKorea,-,07.20.2021,52.5%
1711,doria1611,4371,8550,TeamKorea,-,06.27.2021,51.1%
2056,windcirclet,3578,7022,TeamKorea,Korea,07.20.2021,51.0%
1844,jkl3188,4087,8174,TeamKorea,Korea,06.20.2021,50.0%
2581,piraten1,2691,5445,TeamKorea,-,07.20.2021,49.4%


In [30]:
# HBZ联合舰队
fleet_power('HBZ联合舰队')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
231,cj1230,13495,24147,HBZ联合舰队,-,07.20.2021,55.9%
2313,vincent379,3064,5787,HBZ联合舰队,-,07.17.2021,52.9%


In [31]:
# 乾坤舰队
fleet_power('乾坤舰队')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
472,tjdy,10136,17763,乾坤舰队,-,07.21.2021,57.1%
10007,hundun01,3541,6388,乾坤舰队,-,07.21.2021,55.4%
2269,ninohui218,3129,5883,乾坤舰队,-,07.21.2021,53.2%
843,zhkim72,7514,14362,乾坤舰队,-,07.20.2021,52.3%
2298,oxoxo,3088,5942,乾坤舰队,-,07.20.2021,52.0%
1120,qk015,6214,11998,乾坤舰队,-,07.16.2021,51.8%
1717,sujian521,4354,8517,乾坤舰队,-,07.20.2021,51.1%
2719,hrmd,2498,5017,乾坤舰队,-,07.21.2021,49.8%
1981,miao1974310,3776,7857,乾坤舰队,-,07.20.2021,48.1%


In [32]:
# 皇家零零七
fleet_power('皇家零零七')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
761,kpwlt0013,8040,14519,皇家零零七,情谊,07.11.2021,55.4%
2359,bzsd,3000,5463,皇家零零七,情谊,07.20.2021,54.9%
1809,huizi1616,4149,7743,皇家零零七,情谊,07.19.2021,53.6%
709,xiaonan2018,8320,15725,皇家零零七,情谊,07.20.2021,52.9%
916,sata100,7129,13494,皇家零零七,情谊,07.21.2021,52.8%
2584,ziri2121,2686,5183,皇家零零七,-,07.20.2021,51.8%
1632,huilangooo,4557,8819,皇家零零七,情谊,06.24.2021,51.7%
1295,jian2524,5585,11102,皇家零零七,情谊,07.19.2021,50.3%
1605,ahuan07,4620,9191,皇家零零七,情谊,07.21.2021,50.3%


In [33]:
# 拆迁办
fleet_power('拆迁办')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
960,hyz980517,6902,11229,拆迁办,Logistics,06.16.2021,61.5%
2089,liu001002,3486,5794,拆迁办,-,06.29.2021,60.2%
2559,nulei01,2718,5180,拆迁办,Logistics,07.20.2021,52.5%
617,linhai12,8861,16930,拆迁办,荣耀,07.20.2021,52.3%
2478,gt3000,2825,5415,拆迁办,朱雀,07.20.2021,52.2%
816,hw0871,7687,14769,拆迁办,朱雀,07.17.2021,52.0%
938,pingtouge,6986,13444,拆迁办,玄武,07.20.2021,52.0%
2494,ridgycs,2802,5414,拆迁办,玄武,07.19.2021,51.8%
788,baozhu001,7860,15213,拆迁办,朱雀,07.20.2021,51.7%


In [59]:
# 巅峰舰队
fleet_power('巅峰舰队')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
2241,shuyang66,3183,5974,巅峰舰队,-,07.19.2021,53.3%
529,shuyang88,9581,18617,巅峰舰队,-,07.19.2021,51.5%
10109,laixilaixi888,4290,8495,巅峰舰队,-,07.19.2021,50.5%
2327,lufanss,3050,6054,巅峰舰队,-,07.11.2021,50.4%
1139,tmdsb,6101,12143,巅峰舰队,-,07.11.2021,50.2%


In [34]:
# Homeless
fleet_power('-')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1727,baozi77,4332,5611,-,-,07.20.2021,77.2%
247,k435345,13128,18023,-,-,07.04.2021,72.8%
243,greattak,13184,18291,-,-,07.20.2021,72.1%
1205,znfecu,5860,8207,-,勇敢天下,07.20.2021,71.4%
1149,magey2,6049,8957,-,-,05.10.2021,67.5%
467,josuuu,10185,15206,-,-,07.19.2021,67.0%
584,laopu,9140,13715,-,-,07.21.2021,66.6%
889,fs4025paz,7249,10893,-,-,07.20.2021,66.5%
328,berserker120,11751,18068,-,-,07.21.2021,65.0%


In [35]:
as.list(unique(nf_data['fleet']))

$fleet
 [1] "BAMF FLEET"       "Russian Alliance" "兔子.CHN."        "-"               
 [5] "Ostseepiraten"    "HELL BOYS"        "TEAM PRO BOTS"    "Imperium"        
 [9] "Jedi Fleet"       "FreeThaiMoveMent" "Armada Argentina" "Valhalla"        
[13] "Dt. Flottenbund"  "新荣誉"           "Flota Peruana"    "Armored Angels"  
[17] "大风云舰队"       "Reiwa Fleet"      "Flota Chile"      "中华联合舰队"    
[21] "M.F Fleet"        "C.O.H."           "ULTRAS+"          "中国蓝舰队"      
[25] "海风舰队"         "ARMADA CHILENA"   "VNFRIENDS"        "REGALIA"         
[29] "E.D.F"            "HBZ联合舰队"      "Dango Daikazoku"  "深海舰队"        
[33] "War Navy"         "TeamKorea"        "北方舰队"         "乾坤舰队"        
[37] "TURKIYE"          "Great_Sharks"     "Poseidon"         "尖刀连"          
[41] "第四力量"         "巅峰舰队"         "Flota Gallega"    "名將堂舰队"      
[45] "-Killerigel-"     "LEYENDA HISPANA"  "007舰队"          "拆迁办"          
[49] "皇家零零七"       "TeamNF"           "琥 珀"            "首战"            
[53] "=中国蓝="         "ARGONAUTS"        "The First Shock"  "REGIA.MARINA.ITA"
[57] "齐云舰队"         "云融"             "第七联合舰队"     "猎杀2020"        
[61] "荣誉"             "武林堂"           "Dark Side"        "水手集团总公司"  
[65] "烟民"             "I.J.N"            "UN联合舰队"       "娱乐派对"        
[69] "戦斧"             "旗艦聯合"         "Ｖ＆Ｃ海盗团"     "海狼"            
[73] "CRUZ DEL SUR"     "第九宫舰队"       "北京舰队"         "大秦帝国舰队"    
[77] "八一海军"         "铁十字"           "武汉联合舰队"     "德意志帝国海军"  
[81] "SS帝国公海舰队"   "怒海远征舰队"     "Arg-1982"         "兔子.CHN"        
[85] "H_o_D"            "彩虹舰队"         "新荣誉舰队"       "正和舰队"        
[89] "南海舰队"         "鹰击舰队"         "SeaWolves"        "炎黄海军"        
[93] "达瓦里氏冲锋队"   "自由国度"         "第四力量舰队"     "戮虎-未来之光"   
[97] "三国无双"

In [36]:
as.list(unique(nf_data['squad']))

$squad
  [1] "BULLS"            "Nakhimov"         "-"               
  [4] "Sith"             "Crusaders"        "GLORIUS"         
  [7] "MFDM"             "HisThaiMajestyS"  "IMMORTALS"       
 [10] "P_Beneke"         "DooM"             "Veterans"        
 [13] "corsair"          "Legacy"           "BAP VIP"         
 [16] "USHAK0V"          "Titans SQ"        "ZETAS"           
 [19] "ALDEA"            "NERFED"           "中华联合二队"    
 [22] "M.F共和国之鹰"    "ARA SQ Charlie"   "Aggressor"       
 [25] "Sq Charlie"       "Dragons"          "Sagrados"        
 [28] "#Karma"           "LIONS"            "Saintz SQ"       
 [31] "$CHMUTZ"          "Stoertebeker"     "☆ ☆ ☆"           
 [34] "AborzSQ"          "World"            "BAP_SUR"         
 [37] "=OLAYA="          "THE IMMORTALS"    "讨债办"          
 [40] "Loewengeschwader" "O\"Higgins"       "Tigergeschwader" 
 [43] "HUSARES"          "Rock Squad"       ".-Lord-"         
 [46] "pretender"        "Renovation"       "☆ ★ ☆"           
 [49] "-=HAWK=-"         "★ ★ ☆"            "BoB_Soeldner"    
 [52] "_ECHO"            "_ALPHA"           "jancio dywizjon" 
 [55] "Team Pollera"     "Hermit"           "海風奔馳"        
 [58] "Phil Tin Cans"    "Rum_u_Ehre"       "SQ. CHILE"       
 [61] "Dragon"           "Sons of anarchy"  "勇敢天下"        
 [64] "TATRY"            "bangkok"          "SUPREMO"         
 [67] "中华联合一队"     "THOR"             "Thundercats"     
 [70] "Old Boy"          "ARA SQ Delta"     "海風聚義"        
 [73] "Pure Darkness"    "F_Drake"          "Bravery"         
 [76] "雪豹旅"           "Andes"            "Makarov"         
 [79] "深蓝战队"         "★ ★ ★"            "荣耀"            
 [82] "海狼团"           "情谊"             "虎贲军"          
 [85] "远征军"           "Nanox"            "朱雀"            
 [88] "暴击小分队"       "-=☆☆☆=-"          "United Coalition"
 [91] "ARA SQ Bravo"     ""                 "龍城"            
 [94] "玄武"             "TheBestOfBest"    "AYYY LMAO"       
 [97] "海浪"             "深渊"             "Logistics"       
[100] "R.O.C"            "海风之096老炮"    "共和国之鹰1"     
[103] "★ ☆ ★"            "勛章"             "BLACKFLEET"      
[106] "Sq C.O.H."        "Fr.Kiss"          "Hydra"           
[109] "〓★〓"            "=L&G="            "=B.DemonsSharks" 
[112] "REGIA"            "-=★=-"            "Odin"            
[115] "Legion"           "炮灰团"           "海風"            
[118] "BUCETAO"          "ＺＧＬ"           "兄弟盟"          
[121] "特混编队"         "北纬37度"         "Bikers"          
[124] "扬帆起航"         "特洛伊"           "English Squad"   
[127] "=名将堂="         "北方天骄"         "Cheewooda"       
[130] "ARA SQ Mercs"     "Nuke From Orbit"  "Kyojano"         
[133] "〓☆〓"            "海魂"             "Korea"           
[136] "荣誉团"           "=FG="             "Freedom Fighter" 
[139] "Sith Empire"      "LOKI"             "无畏"            
[142] "天下"             "雷霆"             "佛系"            
[145] "Ｖ＆Ｃ"           "血豹旅"           "Sea Wolves"      
[148] "Grupo Especial"   "Inter union"      "大秦帝国虎贲队"  
[151] "光荣"             "TeamNF Monitor"   "战舰航母"        
[154] "鸭王"             "名将堂雷神"       "拂晓"            
[157] "名将堂战狼"       "JUICY FRUIT"      "德意志黑人抬棺"  
[160] "德意志王牌战队"   "吾之荣誉即忠诚"   "幻象騎士團"      
[163] "鹰雄团"           "深海"             "YAMATOdama"      
[166] "自由战士"         "中华太平洋舰队"   "帝王"            
[169] "扬威四海"         "Fist"             "大风云"          
[172] "USSLaJIWaJ"       "Niko nii~"        "南海舰队"        
[175] "Juliamo"          "Ushakov"          "ARDITI"          
[178] "孤军"             "专业_批发核弹头"  "天驕"            
[181] "F.C.J"            "大秦雄鹰战队"     "AsturSpain"      
[184] "汁男"             "Argentina"        "八一海军"        
[187] "Armored Navy"     "颶風"             "丽春院"          
[190] "统帅堂"           "海上雄狮"         "Team Germany"    
[193] "奉天"             "〓潇★湘〓"        "广东舰队古惑仔"  
[196] "银河舰队"         "锦帆"             "南宁战队"        
[199] "SS帝国公海舰队"   "龙旗飘飘"         "恶魔之魂"        
[202] "彩虹舰队"         "EuropeanCup"      "Jolly_Roger"     
[

In [57]:
# M.F共和国之鹰
squad_power('M.F共和国之鹰')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
69,huitainan_a,19677,29482,M.F Fleet,M.F共和国之鹰,07.20.2021,66.7%
630,z11285208,8757,13595,M.F Fleet,M.F共和国之鹰,07.20.2021,64.4%
10009,gongyun,12235,20248,M.F Fleet,M.F共和国之鹰,07.20.2021,60.4%
1013,fireball194,6660,11898,M.F Fleet,M.F共和国之鹰,07.19.2021,56.0%
10068,yufengauto,4764,9020,M.F Fleet,M.F共和国之鹰,07.20.2021,52.8%
1053,suqi918,6450,12241,M.F Fleet,M.F共和国之鹰,07.16.2021,52.7%
10047,yapoo86,3041,5846,M.F Fleet,M.F共和国之鹰,07.20.2021,52.0%
2252,xx7890033,3158,6161,M.F Fleet,M.F共和国之鹰,07.20.2021,51.3%
1523,ljguohao,4864,9647,M.F Fleet,M.F共和国之鹰,07.20.2021,50.4%


In [37]:
# 芜湖连坑
squad_power('兄弟盟')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1357,rus1976,5376,11081,-,兄弟盟,07.20.2021,48.5%
1444,xc87995348,5084,11073,-,兄弟盟,07.20.2021,45.9%
2867,yd836187854,2328,5378,-,兄弟盟,07.20.2021,43.3%
3052,rb836187854,2112,5242,-,兄弟盟,07.20.2021,40.3%
